<a href="https://colab.research.google.com/github/s-c-soma/AdvanceDeeplearning-CMPE-297/blob/master/Assignment_3/ExtraCredit_Assignment_3_Part_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3 Part A

## Imports

In [ ]:
import tensorflow as tf

from tensorflow.keras import models
from tensorflow.keras import layers

tf.random.set_seed(666)

## Load data

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train = X_train/255.
X_test = X_test/255.



4423680/4422102 [==============================] - 0s 0us/step


((60000, 28, 28), (10000, 28, 28), (60000,), (10000,))

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((60000, 28, 28), (10000, 28, 28), (60000,), (10000,))

## Preprocessing

In [ ]:
X_train = X_train.astype("float32").reshape(-1, 28, 28, 1)
X_test = X_test.astype("float32").reshape(-1, 28, 28, 1)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1), (60000,), (10000,))

## Model

In [ ]:
def build_teacher_model():
    model = models.Sequential()
    model.add(layers.Conv2D(16, (5, 5), activation="relu", input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(32, (5, 5), activation="relu"))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation="relu"))
    model.add(layers.Dense(10))
    loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    optimizer = tf.keras.optimizers.Adam()
    model.compile(loss=loss_func, optimizer=optimizer, metrics=["accuracy"])
    return model

## Fit Model

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(100).batch(64)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(64)


teacher_model = build_teacher_model()
teacher_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 16)        416       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 32)          12832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
dropout (Dropout)            (None, 4, 4, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               6

In [ ]:
teacher_model.fit(train_dataset, validation_data=test_dataset,epochs=20)

Epoch 1/20
938/938 [==============================] - 3s 3ms/step - loss: 0.5603 - accuracy: 0.7962 - val_loss: 0.4060 - val_accuracy: 0.8545
Epoch 2/20
938/938 [==============================] - 3s 3ms/step - loss: 0.3741 - accuracy: 0.8643 - val_loss: 0.3902 - val_accuracy: 0.8586
Epoch 3/20
938/938 [==============================] - 3s 3ms/step - loss: 0.3292 - accuracy: 0.8798 - val_loss: 0.3440 - val_accuracy: 0.8741
Epoch 4/20
938/938 [==============================] - 3s 3ms/step - loss: 0.2990 - accuracy: 0.8906 - val_loss: 0.3324 - val_accuracy: 0.8793
Epoch 5/20
938/938 [==============================] - 3s 3ms/step - loss: 0.2779 - accuracy: 0.8978 - val_loss: 0.3204 - val_accuracy: 0.8833
Epoch 6/20
938/938 [==============================] - 3s 3ms/step - loss: 0.2622 - accuracy: 0.9031 - val_loss: 0.2912 - val_accuracy: 0.8973
Epoch 7/20
938/938 [==============================] - 3s 3ms/step - loss: 0.2491 - accuracy: 0.9066 - val_loss: 0.2929 - val_accuracy: 0.8930
Epoch 

## Evaluate Model

In [ ]:
evaluate_1=teacher_model.evaluate(test_dataset)[1]*100

157/157 [==============================] - 0s 2ms/step - loss: 0.2875 - accuracy: 0.9037


In [ ]:
print("Test accuracy: {:.2f}".format(evaluate_1))


Test accuracy: 90.37


In [ ]:
teacher_model.save_weights("teacher_model.h5")

## Student Model

In [ ]:
from tensorflow.keras.layers import Input, Dense
from tensorflow import nn

In [ ]:
def build_student_model():
    model = models.Sequential()
    model.add(Input(shape=(28, 28, 1)))
    model.add(layers.Flatten())
    model.add(Dense(48, activation="relu"))
    model.add(Dense(10))    
    return model

In [ ]:
from tensorflow.compat.v1.losses import softmax_cross_entropy
from tensorflow.keras.metrics import *
from tensorflow.keras.optimizers import *

In [ ]:
def get_kd_loss(teacher_log,student_log,temp=0.5):
    teacher_probs = nn.softmax(teacher_log / temp)
    kd_loss = softmax_cross_entropy(teacher_probs, student_log/ temp, temp**2)
    return kd_loss

In [ ]:
student_model = build_student_model()
optimizer = Adam(learning_rate=0.01)


train_loss = Mean(name="train_loss")
valid_loss = Mean(name="test_loss")

train_acc = SparseCategoricalAccuracy(name="train_acc")
valid_acc = SparseCategoricalAccuracy(name="valid_acc")

## Train Model

In [ ]:
def train_model(images, labels, teacher_model,student_model, optimizer, temp):
    teacher_log = teacher_model(images)
    with tf.GradientTape() as tape:
        student_log = student_model(images)
        loss = get_kd_loss(teacher_log, student_log, temp)
    
    gradients = tape.gradient(loss, student_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, student_model.trainable_variables))

    train_loss(loss)
    train_acc(labels, nn.softmax(student_log))